### 1. Data Preparation

Data Cleaning and Preprocessing for Analysis -

* Are there any `null values or outliers`? How will you wrangle/handle them?
* Are there any `useful variables` that you can engineer with the given data?
* Do you notice any `patterns or anomalies` in the data? Can you plot them?

In [122]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import time
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sb

from scipy import stats
from scipy.stats import skew
# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

# set up to view all the info of the columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline

In [123]:
#load data
path = 'D:/OneDrive - Northeastern University/Jupyter Notebook/Data Science Projects/CRM-Analysis-for-Marketing-data/marketing data/marketing_data.csv'
#path = '/Users/adityaagarwal/Library/CloudStorage/OneDrive-NortheasternUniversity/Jupyter Notebook/Projects/CRM-Analysis-for-Marketing-data/marketing data/marketing_data.csv'
df = pd.read_csv(path)
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,1826,1970,Graduation,Divorced,"$84,835.00",0,0,6/16/2014,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,SP
1,1,1961,Graduation,Single,"$57,091.00",0,0,6/15/2014,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,0,1,1,0,CA
2,10476,1958,Graduation,Married,"$67,267.00",0,1,5/13/2014,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,0,0,0,0,US
3,1386,1967,Graduation,Together,"$32,474.00",1,1,5/11/2014,0,10,0,1,0,0,0,1,1,0,2,7,0,0,0,0,0,0,0,AUS
4,5371,1989,Graduation,Single,"$21,474.00",1,0,4/8/2014,0,6,16,24,11,0,34,2,3,1,2,7,1,0,0,0,0,1,0,SP


In [124]:
def basic_info(df):
    print("This dataset has ", df.shape[1], " columns and ", df.shape[0], " rows.")
    print("This dataset has ", df[df.duplicated()].shape[0], " duplicated rows.")
    print(" ")
    print("Descriptive statistics of the numeric features in the dataset: ")
    print(" ")
    print(df.describe())
    print(" ")
    print("Information about this dataset: ")
    print(" ")
    print(df.info())

basic_info(df)

This dataset has  28  columns and  2240  rows.
This dataset has  0  duplicated rows.
 
Descriptive statistics of the numeric features in the dataset: 
 
                 ID   Year_Birth      Kidhome     Teenhome      Recency  \
count   2240.000000  2240.000000  2240.000000  2240.000000  2240.000000   
mean    5592.159821  1968.805804     0.444196     0.506250    49.109375   
std     3246.662198    11.984069     0.538398     0.544538    28.962453   
min        0.000000  1893.000000     0.000000     0.000000     0.000000   
25%     2828.250000  1959.000000     0.000000     0.000000    24.000000   
50%     5458.500000  1970.000000     0.000000     0.000000    49.000000   
75%     8427.750000  1977.000000     1.000000     1.000000    74.000000   
max    11191.000000  1996.000000     2.000000     2.000000    99.000000   

          MntWines    MntFruits  MntMeatProducts  MntFishProducts  \
count  2240.000000  2240.000000      2240.000000      2240.000000   
mean    303.935714    26.302232  

Assessment report: **Quality issues**

- There is a space in front of the income's column name
- There are dollar signs is the values of Income column
- The "Income" column has 23 missing values
- Income's type is string
- Dt_Customer's type is string

In [125]:
df_copy = df.copy()

#####  1. Handling Income Column

In [126]:
df_copy.columns = df_copy.columns.str.strip()
print(df_copy.Income.dtype)
print(df_copy.columns)

object
Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Response', 'Complain', 'Country'],
      dtype='object')


In [127]:
df_copy.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Response', 'Complain', 'Country'],
      dtype='object')

In [128]:
df_copy.Income = df_copy.Income.str.strip('$')
df_copy.Income = df_copy.Income.str.replace(".", "")
df_copy.Income = df_copy.Income.str.replace(",", "")
df_copy.Income = df_copy.Income.str.replace("00 ", "")

In [129]:
df_copy['Income'].sample(5)

1155    56067
1548    40246
1932    80573
305     83917
1635    54693
Name: Income, dtype: object

##### 2. Handling Missing values in Income Predictor Variable and convert to Int datatype

In [130]:
# divide the data into two dataframes: one has income values, and the other doesn't.
have_income = df_copy[df_copy.Income.isnull()==False]
missing_income = df_copy[df_copy.Income.isnull()==True]

In [131]:
print("Number of Customers with income =",have_income.shape[0])
print("Number of customers without income =",missing_income.shape[0])

Number of Customers with income = 2216
Number of customers without income = 24


In [132]:
# Convert the one that has income to int type
have_income.Income = have_income.Income.astype(int)

# give a string value of "0" to missing value, then we can convert it into int type
missing_income.Income = str(have_income.Income.median())

missing_income.Income = missing_income.Income.str.replace(".5", "")
missing_income.Income = missing_income.Income.astype(int)

In [133]:
# combine the data
df_copy = pd.concat([have_income, missing_income])
df_copy.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,1826,1970,Graduation,Divorced,84835,0,0,6/16/2014,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,SP
1,1,1961,Graduation,Single,57091,0,0,6/15/2014,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,0,1,1,0,CA
2,10476,1958,Graduation,Married,67267,0,1,5/13/2014,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,0,0,0,0,US
3,1386,1967,Graduation,Together,32474,1,1,5/11/2014,0,10,0,1,0,0,0,1,1,0,2,7,0,0,0,0,0,0,0,AUS
4,5371,1989,Graduation,Single,21474,1,0,4/8/2014,0,6,16,24,11,0,34,2,3,1,2,7,1,0,0,0,0,1,0,SP


##### 3. Handling Datetime Variables

In [134]:
df_copy.Dt_Customer = pd.to_datetime(df_copy.Dt_Customer)

In [135]:
## Drop Irrelevant Features
df_copy.drop(columns=['ID'], inplace=True)

In [136]:
## Check for missing values
df_copy.isnull().sum()

Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Response               0
Complain               0
Country                0
dtype: int64

##### 4. Handling Duplicate data

In [137]:
# Remove duplicate rows
def duplicates_removal(df):
    before = df.shape[0]
    df = df.drop_duplicates()
    after = df.shape[0]
    print(f"Removed {before - after} duplicate rows.")
    return df

df_copy = duplicates_removal(df_copy)

Removed 47 duplicate rows.


##### 5. Grouping Rare Occuring Categories as "OTHERS"

In [138]:
cat_cols = df_copy.select_dtypes(include=['object']).columns.tolist()

def group_rare_categories(df, col, threshold=0.01):
    freq = df[col].value_counts(normalize=True)
    rare = freq[freq < threshold].index
    print(f"Grouping rare categories in {col}: {list(rare)}")
    df[col] = df[col].replace(rare, 'Other')
    return df

for col in cat_cols:
     df_copy = group_rare_categories(df_copy, col)

Grouping rare categories in Education: []
Grouping rare categories in Marital_Status: ['Alone', 'YOLO', 'Absurd']
Grouping rare categories in Country: ['ME']


##### 6. Outlier detection and treatment
Outlier treatment is performed to cap extreme values in numerical columns to a reasonable range (using the IQR method). Outliers can distort statistical analyses, affect the performance of machine learning models, and lead to misleading insights. By capping outliers:

- We reduce the influence of extreme values on mean, standard deviation, and model coefficients.
- Clustering and segmentation algorithms (like KMeans) become more robust and less sensitive to a few extreme data points.
- Visualizations and summary statistics better reflect the majority of the data.

In the long run, this leads to more stable, generalizable, and interpretable models and analyses, especially for `customer segmentation`, `campaign analysis`, and other downstream tasks.

In [117]:
# Outlier Detection and Treatment (IQR method for numerical columns)
def treat_outliers(df, cols):
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = ((df[col] < lower) | (df[col] > upper)).sum()
        print(f"{col}: {outliers} outliers")
        # Cap outliers
        df[col] = df[col].clip(lower, upper)
    return df

num_cols = df_copy.select_dtypes(include=['int64', 'float64']).columns.tolist()
min_max_before = pd.DataFrame(df_copy[num_cols].describe())

if num_cols:
    df_copy = treat_outliers(df_copy, num_cols)

min_max_after = pd.DataFrame(df_copy[num_cols].describe())

Year_Birth: 3 outliers
Income: 8 outliers
Kidhome: 0 outliers
Teenhome: 0 outliers
Recency: 0 outliers
MntWines: 35 outliers
MntFruits: 227 outliers
MntMeatProducts: 175 outliers
MntFishProducts: 223 outliers
MntSweetProducts: 248 outliers
MntGoldProds: 207 outliers
NumDealsPurchases: 86 outliers
NumWebPurchases: 4 outliers
NumCatalogPurchases: 23 outliers
NumStorePurchases: 0 outliers
NumWebVisitsMonth: 8 outliers
AcceptedCmp3: 163 outliers
AcceptedCmp4: 167 outliers
AcceptedCmp5: 163 outliers
AcceptedCmp1: 144 outliers
AcceptedCmp2: 30 outliers
Response: 334 outliers
Complain: 21 outliers


In [118]:
min_max_after - min_max_before

,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain
count,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mean,0.046429,-372.100893,0.0,0.0,0.0,-1.822768,-4.623214,-16.076786,-5.409598,-5.125000,-4.534152,-0.109375,-0.024107,-0.037946,0.0,-0.017411,-0.072768,-0.074554,-0.072768,-0.064286,-0.013393,-0.149107,-0.009375
std,-0.212344,-4101.851129,0.0,0.0,0.0,-5.540037,-12.516086,-46.898715,-14.267495,-13.740164,-12.239931,-0.393566,-0.131381,-0.188323,0.0,-0.078288,-0.259813,-0.262728,-0.259813,-0.245316,-0.114976,-0.356274,-0.096391
min,39.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.000000,-549249.750000,0.0,0.0,0.0,-268.000000,-118.000000,-1169.000000,-138.500000,-182.000000,-235.500000,-9.000000,-15.000000,-18.000000,0.0,-7.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000


##### 7. Handle Skewness in Data

In [ ]:
skewness_threshold = 1
proportion_positive_threshold = 0.5
skewed_cols = []

for col in num_cols:
    if abs(df_copy[col].skew())> skewness_threshold:
        skewed_cols.append(col)
print(skewed_cols)

for col in skewed_cols:
    proportion_positive = df_copy[col] > 0.0
    if proportion_positive.mean() > proportion_positive_threshold:
        df_copy[col] = np.where(df_copy[col] > 0, np.log1p(df_copy[col]), df_copy[col])
        # Calculate skewness before and after correction
        skewness_before = df_copy[col].skew()
        skewness_after = skew(df_copy[col])
        print(f"Skewness before correction for column {col}: {skewness_before}")
        print(f"Skewness after correction for column {col}: {skewness_after}")
    else:
        # Apply a different skewness correction technique, such as Box-Cox transformation
        df_copy[col] = np.where(df_copy[col] > 0, np.power(df_copy[col], 1/3), df_copy[col])
        # Calculate skewness before and after correction
        skewness_before = df_copy[col].skew()
        skewness_after = skew(df_copy[col])
        print(f"Skewness before correction for column {col}: {skewness_before}")
        print(f"Skewness after correction for column {col}: {skewness_after}")

In [ ]:
# Correlation Analysis (drop highly correlated features)
corr_matrix = df_copy[num_cols].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(to_drop)
#if to_drop:
#    print(f"Dropping highly correlated columns: {to_drop}")
#    df_copy = df_copy.drop(columns=to_drop)

In [140]:
# store the cleaned data to a new file
df_copy.reset_index(drop=True)
df_copy.to_csv('marketing data/clean_marketing_data.csv', index=False)